# Get data and packages

In [1]:
%%capture
!pip install vaex-core vaex-hdf5
!pip install -U numpy

In [3]:
!aws s3 cp s3://xdss-public-datasets/demos/taxi_1B.hdf5 ../datasets/taxi_1B.hdf5

download: s3://xdss-public-datasets/demos/taxi_1B.hdf5 to ../datasets/taxi_1B.hdf5g    g 


## Prep benchmarks

In [1]:
import gc
from src.benchmarks_utils import benchmark, get_results
from src.vaex_utils import *
from src.config import repetitions

name = 'vaex'
data_path = '../datasets/taxi_1B.hdf5'
results_path = f"../results/{name}_1b.csv"
benchmarks = {}
print(f"test for {repetitions} repetitions")

test for 1 repetitions


# Benchmark

In [2]:
# Load data
data = read_file(data_path=data_path)
data['pickup_hour'] = data.pickup_datetime.dt.hour
print(f"size: {data.shape[0]} with {data.shape[1]} columns")

size: 1173057927 with 19 columns


In [1]:
data

NameError: name 'data' is not defined

In [3]:
benchmarks['read_file']= benchmark(read_file, df=data, data_path=data_path, repetitions=repetitions)
benchmarks['mean']= benchmark(mean, data, repetitions=repetitions)
benchmarks['standard deviation']= benchmark(standard_deviation, data, repetitions=repetitions)
benchmarks['sum columns']= benchmark(sum_columns, data, repetitions=repetitions)
benchmarks['product columns']= benchmark(product_columns, data, repetitions=repetitions)
benchmarks['arithmetic operation']= benchmark(complicated_arithmetic_operation, data, repetitions=repetitions)
benchmarks['value counts']= benchmark(value_counts, data, repetitions=repetitions)
benchmarks['groupby statistics']= benchmark(groupby_statistics, data, repetitions=repetitions)
benchmarks['filter']= benchmark(filter_data, data, repetitions=repetitions)
print(f"cleaned {gc.collect()} mb")
benchmarks['join'] = benchmark(join, data, repetitions=repetitions, other=groupby_statistics(data))
print(f"Done benchmarks on all data")

# filtered
filterd = filter_data(data)
del data

print(f"Prepare filtered data and deleted {gc.collect()} MB")
benchmarks['filtered mean'] = benchmark(mean, filterd, repetitions=repetitions)
benchmarks['filtered standard deviation'] = benchmark(standard_deviation, filterd, repetitions=repetitions)
benchmarks['filtered sum columns'] = benchmark(sum_columns , filterd, repetitions=repetitions)
benchmarks['filtered product_columns'] = benchmark(product_columns , filterd, repetitions=repetitions)
benchmarks['filtered complicated arithmetic operation'] = benchmark(complicated_arithmetic_operation, filterd, repetitions=repetitions)
benchmarks['filtered value counts'] = benchmark(value_counts, filterd, repetitions=repetitions)
benchmarks['filtered groupby statistics'] = benchmark(groupby_statistics, filterd, repetitions=repetitions)
benchmarks['filtered join'] = benchmark(join, filterd, repetitions=repetitions, other=groupby_statistics(filterd))
print(f"Done benchmarks on filterd data")


  #  vendor_id    pickup_datetime                dropoff_datetime                 passenger_count  payment_type      trip_distance    pickup_longitude    pickup_latitude    rate_code    store_and_fwd_flag    dropoff_longitude    dropoff_latitude    fare_amount    surcharge    mta_tax    tip_amount    tolls_amount    total_amount    pickup_hour    sum    product    complicated
  0  VTS          2009-01-04 02:52:00.000000000  2009-01-04 03:02:00.000000000                  1  CASH                       2.63            -73.992             40.7216          nan                   nan             -73.9938             40.6959            8.9          0.5        nan          0                  0            9.4               2    9.9        8.9    0.000127551
  1  VTS          2009-01-04 03:31:00.000000000  2009-01-04 03:38:00.000000000                  3  Credit                     4.55            -73.9821            40.7363          nan                   nan             -73.9558             40.7

In [4]:
results = get_results(benchmarks, name)
results.to_csv(results_path)
results.head()

,vaex
read_file,0.009607
mean,1.157343
standard deviation,2.813296
sum columns,0.708558
product columns,0.659668


In [5]:
!aws s3 cp  ../results/vaex_1b.csv s3://vaex-sagemaker-demo/benchmarks/vaex_1b_results.csv 

upload: ../results/vaex_1b.csv to s3://vaex-sagemaker-demo/benchmarks/vaex_1b_results.csv
